In [1]:
from torch import cuda
import transformers
from langchain.llms import HuggingFacePipeline, OpenAI

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

device = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"

model = transformers.AutoModelForCausalLM.from_pretrained(model_id)

model.eval()

print(f"Model loaded on {device}")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

import torch
stop_list = ['<|user|>', '\n```\n']
stop_token_ids = [torch.LongTensor(tokenizer(x)['input_ids']).to(device) for x in stop_list]

from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # stopping_criteria=stopping_criteria,
    # we pass model parameters here too
    max_new_tokens=128,  # max number of tokens to generate in the output
    repetition_penalty=1.3  # without this output begins repeating
)

llm = HuggingFacePipeline(pipeline=generate_text)

/data/work/turing/chatbot/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded on cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.tools import Tool

In [3]:
loader = PyPDFLoader('/data/study/huggingface.pdf')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [4]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=256, memory_key="chat_history", input_key="question", return_messages=True)

In [5]:
question_generator_template = PromptTemplate.from_template(
    "<|system|>\n"
    "You are an assistant who takes a chat history and a flow up question and "
    "combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}</s>"
    "\n<|user|>\n"
    "Follow up question: {question}</s>"
    "\n<|assistant|>\n"
)

qa_template = PromptTemplate.from_template(
    "<|system|>\n"
    "You are an assistant who reads a context and respond the user question based on it, "
    "if the answer is not in the context do not make it up"
    "\nContext:\n"
    "{context}\n"
    "{chat_history}</s>"
    "\n<|user|>\n"
    "{question}</s>"
    "\n<|assistant|>\n"

)

python_template = PromptTemplate.from_template(
    "<|system|>\n"
    "You are an assistant who write and execute python code to solve the user question "
    "respond always with the code and the execution result only</s>"
    "\n<|user|>\n"
    "{question}</s>"
    "\n<|assistant|>\n"

)

chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=question_generator_template,
    combine_docs_chain_kwargs={'prompt': qa_template},
    verbose=True,
    rephrase_question=False,
    memory=memory
)

python_chain = LLMChain(
    llm=llm,
    verbose=True,
    prompt=python_template,
)

In [6]:
tools = [Tool('Search documents',chain.invoke,'Always try to use this tool first if the answer could be found in the provided context.'),
         Tool('Python code', python_chain.invoke, 'This tool is for execute and write python code to solve some specific problem, use it to find out what a code does too.')]

In [7]:
from langchain.agents import AgentExecutor, create_react_agent

In [8]:
prompt = PromptTemplate.from_template(
'''
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
Thought:{agent_scratchpad}
'''
)

In [10]:
router = OpenAI()
agent = create_react_agent(router, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=5)

In [12]:
import langchain
langchain.debug=True

In [13]:
agent_executor.invoke({'input':'what is huggingface?'})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is huggingface?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is huggingface?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is huggingface?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is huggingface?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "what is huggingface?",
  "intermediate_steps": []
}
[

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[llm/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 7:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Read the documents or execute python code to find out what huggingface is.\nAction: Search documents\nAction Input: huggingface",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 27,
      "total_tokens": 206,
      "prompt_tokens": 179
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 8:parser:ReActSingleInputOutputParser] Entering Parser run with input:
{
  "input": "Read the documents or execute python code to find out what huggingface is.\nAction: Search documents\nAction Input: huggingface"
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 8:parser:ReActSingl

{'input': 'what is huggingface?',
 'output': 'huggingface is a natural language processing library that uses transformers to fine-tune models for language processing tasks. It also has a Context API that allows for easy integration of pre-trained models.'}

In [14]:
agent_executor.invoke({'input':'what is models are hosted there?'})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is models are hosted there?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is models are hosted there?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is models are hosted there?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is models are hosted there?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "what 

{'input': 'what is models are hosted there?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [13]:
agent_executor.invoke({'input':'write a python function to sum two numbers'})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "write a python function to sum two numbers"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "write a python function to sum two numbers",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "write a python function to sum two numbers",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": "write a python function to sum two numbers",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering C

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[llm/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 7:llm:OpenAI] [1.04s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Make a function that takes two numbers and adds them together\nAction: Python code\nAction Input: def sum(a,b): return a+b",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 182,
      "completion_tokens": 28,
      "total_tokens": 210
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 8:parser:ReActSingleInputOutputParser] Entering Parser run with input:
{
  "input": "Make a function that takes two numbers and adds them together\nAction: Python code\nAction Input: def sum(a,b): return a+b"
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 8:parser:ReActSingleInpu

{'input': 'write a python function to sum two numbers',
 'output': 'def sum(a,b): return a+b'}